In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import pickle
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
DATA_PATH = "/content/drive/MyDrive/Colab/"
infile = open(DATA_PATH+'training_samples1.pkl','rb')
dataset = pickle.load(infile)

Mounted at /content/drive


In [ ]:
images = []
labels = []
for sample in dataset:
  images.append(sample['image'])
  labels.append(sample['label'])

In [ ]:
images = np.array(images)
labels = np.array(labels)

In [ ]:
print(images.shape)
print(labels.shape)

(2858, 480, 480, 3)
(2858, 480, 480, 1)


In [ ]:
training_data = tf.data.Dataset.from_tensor_slices((images, labels))
print(training_data)

<_TensorSliceDataset element_spec=(TensorSpec(shape=(480, 480, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(480, 480, 1), dtype=tf.uint8, name=None))>


In [ ]:
training_data = training_data.batch(8)


In [ ]:
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, Concatenate
from keras.models import Model

def unet(input_size=(480,480,3)):
    inputs = Input(input_size)
    rescale = keras.layers.Rescaling(1./255)(inputs)
    # Encoder
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(rescale)
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    # Decoder
    conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2,2))(drop5))
    merge6 = Concatenate(axis=3)([conv4, up6])
    conv6 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv6)

    up7 = Conv2D(256, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2,2))(conv6))
    merge7 = Concatenate(axis=3)([conv3, up7])
    conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv7)

    up8 = Conv2D(128, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2,2))(conv7))
    merge8 = Concatenate(axis=3)([conv2, up8])
    conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv8)

    up9 = Conv2D(64, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2,2))(conv8))
    merge9 = Concatenate(axis=3)([conv1, up9])
    conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)

    outputs = Conv2D(1, 1, activation='sigmoid')(conv9)

    model = Model(inputs=inputs, outputs=outputs)

    return model

model = unet()

In [ ]:
model.compile(optimizer='adam', loss=keras.losses.BinaryFocalCrossentropy(), metrics=['accuracy','BinaryIoU'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 480, 480, 3)]        0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 480, 480, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv2d (Conv2D)             (None, 480, 480, 64)         1792      ['rescaling[0][0]']           
                                                                                                  
 conv2d_1 (Conv2D)           (None, 480, 480, 64)         36928     ['conv2d[0][0]']              
                                                                                              

In [ ]:
history = model.fit(training_data, epochs=10)

Epoch 1/10
358/358 [==============================] - 383s 896ms/step - loss: 0.0560 - accuracy: 0.9804 - binary_io_u: 0.4913
Epoch 2/10
358/358 [==============================] - 308s 858ms/step - loss: 0.0234 - accuracy: 0.9829 - binary_io_u: 0.4915
Epoch 3/10
358/358 [==============================] - 308s 858ms/step - loss: 0.0232 - accuracy: 0.9829 - binary_io_u: 0.4915
Epoch 4/10
358/358 [==============================] - 308s 858ms/step - loss: 0.0227 - accuracy: 0.9829 - binary_io_u: 0.4915
Epoch 5/10
358/358 [==============================] - 308s 858ms/step - loss: 0.0224 - accuracy: 0.9829 - binary_io_u: 0.4915
Epoch 6/10
358/358 [==============================] - 308s 858ms/step - loss: 0.0218 - accuracy: 0.9829 - binary_io_u: 0.4915
Epoch 7/10
358/358 [==============================] - 308s 858ms/step - loss: 0.0215 - accuracy: 0.9829 - binary_io_u: 0.4915
Epoch 8/10
358/358 [==============================] - 307s 858ms/step - loss: 0.0212 - accuracy: 0.9829 - binary_io_u:

In [ ]:
model.save('/content/drive/MyDrive/Colab/unet_lane_detection.keras')

In [ ]:
DATA_PATH1 = "/content/drive/MyDrive/Colab/"
infile = open(DATA_PATH1+'testing_sample.pkl','rb')
test_dataset = pickle.load(infile)

In [ ]:
test_image = []
test_label = []
for x in test_dataset:
  test_image.append(x['image'])
  test_label.append(x['label'])

In [ ]:
test_image = np.array(test_image)
test_labels = np.array(test_label)

In [ ]:
print(test_image.shape)
print(test_labels.shape)

(2782, 480, 480, 3)
(2782, 480, 480, 1)


In [ ]:
testing_data = tf.data.Dataset.from_tensor_slices((test_image, test_labels))
print(testing_data)

<_TensorSliceDataset element_spec=(TensorSpec(shape=(480, 480, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(480, 480, 1), dtype=tf.uint8, name=None))>


In [ ]:
testing_data = testing_data.batch(8)

In [ ]:
test_result = model.evaluate(testing_data)

348/348 [==============================] - 126s 359ms/step - loss: 0.0125 - accuracy: 0.9870 - binary_io_u: 0.6741


In [ ]:
pred = model.predict(testing_data)


348/348 [==============================] - 112s 323ms/step


0.0

In [ ]:
pred.max(),pred.min()

(0.8290031, 0.0)

In [ ]:
import pickle as pkl
with open(DATA_PATH1 + 'prediction.pkl', 'wb') as f:
      pkl.dump(pred, f)

In [ ]:
print(pred[0].shape)

(480, 480, 1)
